In [1]:
import warnings
warnings.filterwarnings('ignore')
import glob
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
import random

In [2]:
train =pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submission.csv')
population = pd.read_csv('age_gender_info.csv')

1.자격유형의 데이터의 NaN 데이터의 처리
===

In [3]:
train.loc[train.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수


In [4]:
test.loc[test.자격유형.isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [5]:
test.loc[test.단지코드 == 'C2411'] #C2411의 데이터에서 같은 조건의 데이터가 A이므로 마찬가지로 A로 변경

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
193,C2411,962,아파트,경상남도,국민임대,39.43,56,25.0,A,11992000,100720,0.0,2.0,840.0
194,C2411,962,아파트,경상남도,국민임대,39.72,336,25.0,A,11992000,100720,0.0,2.0,840.0
195,C2411,962,아파트,경상남도,국민임대,39.82,179,25.0,A,11992000,100720,0.0,2.0,840.0
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
197,C2411,962,아파트,경상남도,국민임대,51.93,150,25.0,A,21586000,171480,0.0,2.0,840.0


In [6]:
test.loc[(test.단지코드 == 'C2411') & (test.자격유형.isnull()), '자격유형'] = 'A'

In [7]:
test.loc[test.단지코드 == 'C2253'].head(3) #C2253의 데이터에서 같은 조건의 데이터가 C이므로 마찬가지로 C로 변경

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0
259,C2253,1161,아파트,강원도,영구임대,31.32,239,0.0,C,3731000,83020,0.0,2.0,173.0
260,C2253,1161,아파트,강원도,영구임대,31.32,149,0.0,C,3731000,83020,0.0,2.0,173.0


In [8]:
test.loc[(test.단지코드 == 'C2253') & (test.자격유형.isnull()), '자격유형'] = 'C'

2-1.지하철과 관련된 누락된 데이터의 처리
===

train에서는 지하철관련 총 211개의 데이터에서 누락된 부분이 존재

1. 층청남도에 114개의 데이터 누락(광주에만 도시철도가 존재하며 데이터의 값이 0)

2. 경상남도에 4개의 데이터 누락(부산에만 도시철도가 존재하며 경상남도 데이터의 경우 값이 0)

3. 대전광역시에 93개의 데이터 누락(대전광역시의 경우 데이터가 0과 1인 경우 혼재)  
->대전광역시에 관한 노선에 대한 누락값 채우기는 고민해야함 

In [9]:
train.loc[train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()].head(3) #211개

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
94,C1312,518,아파트,충청남도,국민임대,39.72,60,12.0,A,17460000,122210,NaN,3.0,527.0,359.0
95,C1312,518,아파트,충청남도,국민임대,39.98,89,12.0,A,17460000,122210,NaN,3.0,527.0,359.0
96,C1312,518,아파트,충청남도,국민임대,41.55,225,12.0,A,19954000,130940,NaN,3.0,527.0,359.0


In [10]:
train.loc[(train.지역 == '충청남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull())].shape

(114, 15)

In [11]:
train.loc[(train.지역 == '경상남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull())].shape

(4, 15)

In [12]:
train.loc[(train.지역 == '대전광역시') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull())].shape

(93, 15)

In [13]:
train.loc[(train.지역 == '충청남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = 0
train.loc[(train.지역 == '경상남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = 0

In [14]:
train.loc[(train.지역 == '대전광역시') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = 0

2-2.버스 정류장과 관련된 누락된 데이터의 처리
===

train에서는 버스 정류장관련 총 4개의 데이터에서 누락된 부분이 존재  
전부 경상남도 이므로 경상남도 지역의 데이터로 처리하는 것이 알맞아 보임
데이터 형태 확인 시 3.0의 값이 중앙값이므로 행당 데이터를 누락된 데이터에 채워주기로 결정 -> 추후 변경가능

In [15]:
train.loc[train['도보 10분거리 내 버스정류장 수'].isnull()] #4개

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2315,C1649,575,아파트,경상남도,공공임대(10년),74.97,80,15.0,A,46000000,456000,0.0,NaN,1066.0,855.0
2316,C1649,575,아파트,경상남도,공공임대(10년),84.95,124,15.0,A,57000000,462000,0.0,NaN,1066.0,855.0
2317,C1649,575,아파트,경상남도,공공임대(10년),84.96,289,15.0,A,57000000,462000,0.0,NaN,1066.0,855.0
2318,C1649,575,아파트,경상남도,공공임대(10년),84.98,82,15.0,A,57000000,462000,0.0,NaN,1066.0,855.0


In [16]:
train.loc[(train.지역 == '경상남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].notnull()), '도보 10분거리 내 버스정류장 수'].median() 

3.0

In [17]:
train.loc[(train.지역 == '경상남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].notnull())].describe() 

,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
count,369.000000,369.000000,369.000000,369.000000,369.0,365.000000,369.000000,369.000000
mean,695.644986,42.194092,98.260163,11.560976,0.0,3.824658,475.739837,414.238482
std,281.523748,23.425949,116.087076,11.160255,0.0,3.500698,324.380048,319.843563
min,212.000000,14.170000,1.000000,0.000000,0.0,1.000000,65.000000,31.000000
25%,481.000000,31.840000,1.000000,2.000000,0.0,2.000000,166.000000,120.000000
50%,656.000000,36.880000,60.000000,7.000000,0.0,3.000000,445.000000,398.000000
75%,840.000000,46.920000,137.000000,21.000000,0.0,4.000000,658.000000,616.000000
max,1696.000000,317.170000,728.000000,49.000000,0.0,20.000000,1648.000000,1359.000000


In [18]:
train.loc[train['도보 10분거리 내 버스정류장 수'].isnull(), '도보 10분거리 내 버스정류장 수'] = 3.0

In [19]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)      0
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
등록차량수                             0
dtype: int64

3.임대보증금 및 임대료에 관한 누락된 데이터의 처리
===
1. nan 데이터의 경우  
관련 자료를 확인하게 되면 일부 자격유형의 경우 임대보증금과 임대료를 요구하지 않는 것으로 보임
해당 하는 경우는 D 유형에서 발생. 따라서 이 경우 0으로 대체

2. - 데이터의 경우  
- 데이터의 유형으로 보이는 것은 H 와 K 유형인데 해당 자격을 가지는 다른 데이터를 확인할 경우 임대보증금과 임대료가 적혀있는 것을 확인할 수 있음 따라서, 해당 데이터를 채우기 위하여 별도의 과정이 필요할 것으로 보임.(일단 이번 과정에서는 0으로 대체)

In [20]:
train.loc[(train.임대건물구분 != '상가') & (train.임대보증금.isnull())] 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0


In [21]:
train.loc[(train.임대건물구분 != '상가') & (train.임대료.isnull())] 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2331,C1350,1401,아파트,대전광역시,공공분양,74.94,317,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2332,C1350,1401,아파트,대전광역시,공공분양,74.94,137,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2333,C1350,1401,아파트,대전광역시,공공분양,74.94,22,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2334,C1350,1401,아파트,대전광역시,공공분양,84.94,164,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2335,C1350,1401,아파트,대전광역시,공공분양,84.94,19,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2336,C1350,1401,아파트,대전광역시,공공분양,84.96,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0
2337,C1350,1401,아파트,대전광역시,공공분양,84.97,26,2.0,D,NaN,NaN,0.0,6.0,1636.0,2315.0


In [22]:
train.loc[train.임대보증금.isnull()] = 0 
train.loc[train.임대료.isnull()] = 0 

test.loc[test.임대보증금.isnull()] = 0 
test.loc[test.임대료.isnull()] = 0 

In [23]:
train.loc[(train.임대보증금 == '-')] 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2548,C1326,1934,아파트,부산광역시,국민임대,24.79,104,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2549,C1326,1934,아파트,부산광역시,국민임대,26.83,590,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2550,C1326,1934,아파트,부산광역시,국민임대,37.70,464,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2551,C1326,1934,아파트,부산광역시,국민임대,46.94,304,43.0,H,-,-,0.0,4.0,1670.0,1153.0
2680,C1786,480,아파트,강원도,행복주택,16.91,156,25.0,K,-,-,0.0,3.0,338.0,345.0
2681,C1786,480,아파트,강원도,행복주택,26.90,136,25.0,K,-,-,0.0,3.0,338.0,345.0
2682,C1786,480,아파트,강원도,행복주택,26.90,72,25.0,K,-,-,0.0,3.0,338.0,345.0
2683,C1786,480,아파트,강원도,행복주택,26.90,24,25.0,K,-,-,0.0,3.0,338.0,345.0
2906,C2186,924,아파트,대구광역시,국민임대,29.17,238,0.0,H,-,-,0.0,8.0,664.0,744.0


In [24]:
train.loc[(train.자격유형 == 'H') & (train.지역 =='대구광역시')] 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2906,C2186,924,아파트,대구광역시,국민임대,29.17,238,0.0,H,-,-,0.0,8.0,664.0,744.0
2907,C2186,924,아파트,대구광역시,국민임대,29.34,38,0.0,H,-,-,0.0,8.0,664.0,744.0
2908,C2186,924,아파트,대구광역시,국민임대,37.43,220,0.0,H,-,-,0.0,8.0,664.0,744.0
2909,C2186,924,아파트,대구광역시,국민임대,46.51,188,0.0,H,25981000,248630,0.0,8.0,664.0,744.0


In [25]:
train.loc[(train.자격유형 == 'K') & (train.지역 =='강원도')] 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2680,C1786,480,아파트,강원도,행복주택,16.91,156,25.0,K,-,-,0.0,3.0,338.0,345.0
2681,C1786,480,아파트,강원도,행복주택,26.90,136,25.0,K,-,-,0.0,3.0,338.0,345.0
2682,C1786,480,아파트,강원도,행복주택,26.90,72,25.0,K,-,-,0.0,3.0,338.0,345.0
2683,C1786,480,아파트,강원도,행복주택,26.90,24,25.0,K,-,-,0.0,3.0,338.0,345.0


In [26]:
train.loc[(train.임대보증금 == '-')] = 0
train.loc[(train.임대료 == '-')] = 0

test.loc[(test.임대보증금 == '-')] = 0
test.loc[(test.임대료 == '-')] = 0

In [27]:
train.info() #데이터 타입 체크

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2952 non-null   object 
 10  임대료                           2952 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2952 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2952 non-null   float64
 13  단지내

In [28]:
train.임대보증금 = train.임대보증금.astype('float64')
train.임대료 = train.임대료.astype('float64')

In [29]:
train['임대건물구분'] = train['임대건물구분'].astype('category').cat.codes
train['지역'] = train['지역'].astype('category').cat.codes
train['공급유형'] = train['공급유형'].astype('category').cat.codes
train['자격유형'] = train['자격유형'].astype('category').cat.codes

In [30]:
test.임대보증금 = test.임대보증금.astype('float64')
test.임대료 = test.임대료.astype('float64')

In [31]:
test['임대건물구분'] = test['임대건물구분'].astype('category').cat.codes
test['지역'] = test['지역'].astype('category').cat.codes
test['공급유형'] = test['공급유형'].astype('category').cat.codes
test['자격유형'] = test['자격유형'].astype('category').cat.codes

In [32]:
train.info() #데이터 타입 체크

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   int8   
 3   지역                            2952 non-null   int8   
 4   공급유형                          2952 non-null   int8   
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   int8   
 9   임대보증금                         2952 non-null   float64
 10  임대료                           2952 non-null   float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2952 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2952 non-null   float64
 13  단지내

In [33]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   int8   
 3   지역                            1022 non-null   int8   
 4   공급유형                          1022 non-null   int8   
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1022 non-null   int8   
 9   임대보증금                         1022 non-null   float64
 10  임대료                           1022 non-null   float64
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  1002 non-null   float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

In [34]:
train_x = train.drop(['등록차량수', '단지코드'], axis = 1)
train_y = train['등록차량수']

test_x = test.drop(['단지코드'], axis = 1)

In [35]:
train = train.drop(['단지코드'], axis = 1)

In [36]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['등록차량수']):
    folds.append((train_idx, valid_idx))

In [37]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['등록차량수'],axis=1).iloc[train_idx].values, train.drop(['등록차량수'],axis=1).iloc[valid_idx].values,\
                                         train['등록차량수'][train_idx].values, train['등록차량수'][valid_idx].values 
    lgb = LGBMRegressor(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 855.038	valid_1's l2: 1845.31
[200]	training's l2: 204.801	valid_1's l2: 1140.33
[300]	training's l2: 78.1096	valid_1's l2: 954.777
[400]	training's l2: 36.4264	valid_1's l2: 873.448
[500]	training's l2: 19.815	valid_1's l2: 836.455
[600]	training's l2: 11.6582	valid_1's l2: 816.871
[700]	training's l2: 6.95782	valid_1's l2: 804.465
[800]	training's l2: 4.38528	valid_1's l2: 795.608
[900]	training's l2: 2.86259	valid_1's l2: 790.731
[1000]	training's l2: 1.88757	valid_1's l2: 787.005
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 1.88757	valid_1's l2: 787.005


====================================2============================================
Training until validation scores don't improve for 30 rounds
[100]	training's l2: 930.032	valid_1's l2: 3045.96
[200]	training's l2: 242.09	valid_1's l2: 2083.7
[3

In [38]:
submit.iloc[:,1:]=0
pred_y = test['단지내주차면수']
pred_y = 0
for fold in range(5):
    pred_y += lgb_models[fold].predict(test_x)/5

In [39]:
pred_y = pd.DataFrame(pred_y)

In [40]:
pred_y['단지코드'] = test['단지코드']

In [41]:
pred_y

,0,단지코드
0,763.693661,C1072
1,784.052987,C1072
2,771.646273,C1072
3,769.544270,C1072
4,782.560116,C1072
...,...,...
1017,-0.000100,0
1018,217.209452,C2189
1019,213.738151,C2189
1020,226.200411,C2189


In [42]:
result = pred_y.groupby('단지코드').sum()

In [43]:
value = pred_y.groupby('단지코드').count()

In [44]:
result

,0
단지코드,
0,-0.018614
C1003,1935.626039
C1006,934.204620
C1016,3200.682273
C1019,1179.615152
...,...
C2653,3836.664189
C2675,8885.419761
C2676,530.239541


In [46]:
result = result/value

In [47]:
result

,0
단지코드,
0,-0.000100
C1003,241.953255
C1006,233.551155
C1016,800.170568
C1019,294.903788
...,...
C2653,959.166047
C2675,1110.677470
C2676,176.746514


In [48]:
pred_list = result.index.tolist()
for i in submit['code']:
    if(i in pred_list):
        submit.loc[submit.code == i, 'num'] = result.loc[result.index == i, 0][0]

In [49]:
submit

,code,num
0,C1072,781.337274
1,C1128,1519.259827
2,C1456,514.691142
3,C1840,488.247689
4,C1332,1126.507931
...,...,...
145,C2456,218.568857
146,C1266,532.516841
147,C2152,0.000000
148,C1267,495.741932


In [50]:
submit.to_csv('20210706_1.csv', index = False)